In [1]:
!pip install "torch==2.4.0"
!pip install "peft==0.13.2"
!pip install "trl==0.11.1"
!pip install "transformers==4.44.2"
!pip install "bitsandbytes==0.44.1"
!pip install "accelerate==0.34.2"
!pip install "huggingface_hub==0.25.1"
!pip install "unsloth[cu121-ampere-torch240] @ git+https://github.com/unslothai/unsloth.git"     

import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 90.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 99.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 101.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 114.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 108.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 112.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 119.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 120.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvtx-cu12
    Found existing installation: nvidia-nvtx-cu12 12.4.99
    Uninstalling nvidia-nvtx-cu12-12.4.99:
      Successfully uninstalled nvidia-nvtx-cu12-12.4.99
  Attempt

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-32B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.2: Fast Qwen2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.254 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/280k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=52,
    use_rslora=False,
    loftq_config=None
)

Unsloth 2024.10.2 patched 64 layers with 0 QKV layers, 64 O layers and 64 MLP layers.


In [4]:
prompt = """
### Instruction:
Вы — опытный ментор по программированию, помогающий студентам учиться на своих ошибках. Ваша задача — давать студенту полезные советы и направлять его, не предоставляя готового решения задачи. Когда студент показывает код, вы должны:
Никогда не давать готовое решение задачи.
Указать на ошибку или недочет, если они есть.
Объяснить, в чем состоит проблема, и предложить направление для исправления.
При необходимости объяснять концепции, которые могут помочь студенту найти решение самостоятельно.
Объясните, что может быть не так в его решении, используя за основу одну или несколько из следующих фраз:
- Обратите внимание на неверный...
- ...необходимо проверить, что...
- Вы забыли поставить...
- Необходимо использовать...
- Вы некорректно...
- Проверьте написание...
- В данном случае не нужно...
- ...неверный синтаксис...
- Ваш код охватывает не все возможные случаи...
- Попробуйте дополнить...
- Вы использовали неверную...
- Ошибка при обращении к...
- Проверьте, что...
- Ваш код использует неверный...

### Input:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

In [5]:
import pandas as pd
import pickle

In [6]:
!pip install openpyxl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
train_solutions = pd.read_excel('/workspace/train/solutions.xlsx')
train_tasks = pd.read_excel('/workspace/train/tasks.xlsx')
train_tests = pd.read_excel('/workspace/train/tests.xlsx')

synth = pd.read_csv('/workspace/train_generated.csv')

with open('/workspace/NEW_hints_synth_total.pkl', 'rb') as f:
    synth_hints = pickle.load(f)

In [8]:
def combine_data(solutions_, tasks_, tests_):
    solutions, tasks, tests = solutions_.copy(), tasks_.copy(), tests_.copy()
    
    grouped_tests = tests.groupby('task_id').apply(
        lambda x: "; ".join([
            f"number: {row['number']}, type: {row['type']}, input: {row['input']}, output: {row['output']}"
            for _, row in x.iterrows()
        ])
    ).reset_index()
    grouped_tests = grouped_tests.rename(columns={0: 'tests'})
    
    merged_df = pd.merge(solutions, tasks, left_on='task_id', right_on='id', suffixes=('_solution', '_task'))
    final_df = pd.merge(merged_df, grouped_tests, how='left', on='task_id')
    
    return final_df

In [10]:
train = combine_data(train_solutions, train_tasks, train_tests)

/tmp/ipykernel_99/1588904424.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_tests = tests.groupby('task_id').apply(


In [11]:
train_text = train[['description', 'author_solution', 'student_solution', 'author_comment', 'tests']]

synth['author_comment'] = synth_hints
synth_text = synth[['description', 'author_solution', 'student_solution', 'author_comment', 'tests']]

train_text = pd.concat([train_text, synth_text], axis=0).sample(frac=1, random_state=52).reset_index(drop=True)

In [12]:
input_template = '''
Условие задачи: {description}

Эталонное решение: {author_solution}

Решение студента: {student_solution}
'''

output_template = '''
Подсказка: {author_comment}
'''

In [14]:
train_df = pd.DataFrame({
    'input': train_text.apply(lambda row: input_template.format(
        description=row['description'],
        author_solution=row['author_solution'],
        student_solution=row['student_solution']
    ), axis=1),
    'output': train_text.apply(lambda row: output_template.format(
        author_comment=row['author_comment']
    ), axis=1)
})

In [15]:
# from sklearn.model_selection import train_test_split

# train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=52)

In [16]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
# val_dataset = Dataset.from_pandas(val_df)

train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
# val_dataset = val_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/1455 [00:00<?, ? examples/s]

In [17]:
# from transformers import AutoModel, AutoTokenizer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# bert_model_name = "DeepPavlov/rubert-base-cased-sentence"
# bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
# bert_model = AutoModel.from_pretrained(bert_model_name)

In [18]:
# def get_bert_embeddings(sentence):
#     inputs = bert_tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=256)
#     with torch.no_grad():
#         outputs = bert_model(**inputs)
#         embedding = outputs.last_hidden_state[:, 0, :].squeeze()
#     return embedding

In [19]:
# def compute_cossim(preds, labels):
#     pred_embeddings = torch.stack([get_bert_embeddings(pred) for pred in preds])
#     label_embeddings = torch.stack([get_bert_embeddings(label) for label in labels])
    
#     pred_embeddings_np = pred_embeddings.squeeze().numpy()
#     label_embeddings_np = label_embeddings.squeeze().numpy()
    
#     cossim_scores = cosine_similarity(pred_embeddings_np, label_embeddings_np)
#     mean_cossim = np.mean(np.diag(cossim_scores))
    
#     return mean_cossim

In [20]:
# def compute_metrics(eval_preds):
#     preds, labels = eval_preds

#     decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in preds]
#     decoded_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]
    
#     cossim_metric = compute_cossim(decoded_preds, decoded_labels)
    
#     return {"cossim": cossim_metric}

In [21]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

args = TrainingArguments(
    report_to='none',
    num_train_epochs=1,
    per_device_train_batch_size=2,
#     per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    learning_rate=2e-4,
    fp16=(not is_bfloat16_supported()),
    bf16=(is_bfloat16_supported()),
#     eval_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=52,
    output_dir="outputs" # !!! менять путь чтоб не перезаписывались чекпоинты
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
#     eval_dataset=val_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=args,
#     compute_metrics=compute_metrics
)

Map (num_proc=2):   0%|          | 0/1455 [00:00<?, ? examples/s]

In [24]:
trainer_stats = trainer.train()

**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers and Unsloth!


/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3098: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Step,Training Loss
121,0.070000
122,0.050900
123,0.057700
124,0.035000
125,0.070600
126,0.034000
127,0.037500
128,0.040200
129,0.051100
130,0.038200


In [26]:
!nvidia-smi

Fri Oct 18 10:17:49 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:81:00.0 Off |                    0 |
| N/A   39C    P0             73W /  300W |   32711MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [27]:
from huggingface_hub import login

login()

In [28]:
model.push_to_hub('qwen32b-4bit-lora-newsynth-182steps')

README.md:   0%|          | 0.00/598 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

Saved model to https://huggingface.co/qwen32b-4bit-lora-newsynth-182steps
